In [7]:
import numpy as np
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
URM_path="../data/interactions_and_impressions.csv"
URM_path2="../data/interactions_and_impressions_v3.csv"
URM_all_dataframe = pd.read_csv(URM_path)

URM_all_dataframe.columns = ["UserID", "ItemID", "others","Data"]
URM_all_dataframe=URM_all_dataframe.drop("others",axis=1)
URM_all_dataframe["Data"] = URM_all_dataframe["Data"].replace({0: 1, 1: 0})


URM_all_dataframe2 = pd.read_csv(URM_path2)
URM_all_dataframe2.columns = ["UserID", "ItemID","Data"]
#URM_all_dataframe2 = URM_all_dataframe2[URM_all_dataframe2['Data'] != 0.01]

URM_all_dataframe2 = URM_all_dataframe2.sort_values(by=['UserID', 'ItemID', 'Data'])
URM_all_dataframe = URM_all_dataframe.sort_values(by=['UserID', 'ItemID', 'Data'])

all_users_items=list(set(list(zip(URM_all_dataframe['UserID'],URM_all_dataframe['ItemID']))))
np.random.shuffle(all_users_items)


import random
train_percentage=0.8
user_for_training=random.sample(all_users_items, round(len(all_users_items)*train_percentage))
user_for_validation=list(set(all_users_items)-set(user_for_training))
user_for_training=pd.DataFrame(user_for_training,columns=["UserID","ItemID"])
user_for_validation=pd.DataFrame(user_for_validation,columns=["UserID","ItemID"])
data_train2 = URM_all_dataframe2[URM_all_dataframe2.set_index(["UserID","ItemID"]).index.isin(user_for_training.set_index(["UserID","ItemID"]).index)]
data_train = URM_all_dataframe[URM_all_dataframe.set_index(["UserID","ItemID"]).index.isin(user_for_training.set_index(["UserID","ItemID"]).index)]
#https://stackoverflow.com/questions/54006298/select-rows-of-a-dataframe-based-on-another-dataframe-in-python



data_valid2 = URM_all_dataframe2[URM_all_dataframe2.set_index(["UserID","ItemID"]).index.isin(user_for_validation.set_index(["UserID","ItemID"]).index)]
data_valid = URM_all_dataframe[URM_all_dataframe.set_index(["UserID","ItemID"]).index.isin(user_for_validation.set_index(["UserID","ItemID"]).index)]

import numpy as np
import scipy.sparse as sps


URM_train = sps.csr_matrix((data_train["Data"].values,
                          (data_train["UserID"].values, data_train["ItemID"].values)))
URM_train2 = sps.csr_matrix((data_train2["Data"].values,
                          (data_train2["UserID"].values, data_train2["ItemID"].values)))
URM_valid = sps.csr_matrix((data_valid["Data"].values,
                             (data_valid["UserID"].values, data_valid["ItemID"].values)))
URM_valid2 = sps.csr_matrix((data_valid2["Data"].values,
                             (data_valid2["UserID"].values, data_valid2["ItemID"].values)))
URM_train,URM_train2 ,URM_valid,URM_valid2

/var/folders/fv/ckspmkyx14g_h7sv41m5pjj80000gn/T/ipykernel_13220/1219749331.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  URM_all_dataframe = pd.read_csv(URM_path)


(<41629x24507 sparse matrix of type '<class 'numpy.int64'>'
 	with 1243712 stored elements in Compressed Sparse Row format>,
 <41629x24507 sparse matrix of type '<class 'numpy.float64'>'
 	with 1243704 stored elements in Compressed Sparse Row format>,
 <41629x24507 sparse matrix of type '<class 'numpy.int64'>'
 	with 310928 stored elements in Compressed Sparse Row format>,
 <41629x24507 sparse matrix of type '<class 'numpy.float64'>'
 	with 310928 stored elements in Compressed Sparse Row format>)

In [1]:
from Utils import Reader
from Utils.Evaluator import EvaluatorHoldout
URM_train,URM_train2 ,URM_valid,URM_valid2=Reader.split_train_validation_double()
evaluator_validation=EvaluatorHoldout(URM_valid,cutoff_list=[10])
evaluator_validation2=EvaluatorHoldout(URM_valid2,cutoff_list=[10])


EvaluatorHoldout: Ignoring 1589 ( 3.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 1589 ( 3.8%) Users that have less than 1 test interactions


In [2]:
#1
# BEST MODEL UNTIL NOW _ PROVA

from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train2)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

Similarity column 24507 (100.0%), 3785.72 column/sec. Elapsed time 6.47 sec
Similarity column 41629 (100.0%), 2343.62 column/sec. Elapsed time 17.76 sec
[[-0.09095383 -0.24738513 -0.10779274 ... -0.350219   -0.06725492
  -0.350219  ]
 [-0.350219   -0.350219   -0.350219   ... -0.350219   -0.350219
  -0.350219  ]
 [ 0.69090515  0.50116843  0.70487016 ... -0.15452862 -0.0322994
  -0.28700966]
 ...
 [ 0.07370103 -0.2261711  -0.03897558 ... -0.350219   -0.21208452
  -0.350219  ]
 [ 0.7203803   0.47170183  0.4486557  ... -0.33649695  0.32763025
  -0.00881661]
 [ 0.715159    0.7706233   0.6208966  ... -0.05203089  0.21328293
  -0.02477284]]
[[-0.04483978 -0.03094017  0.00948494 ... -0.04483978  0.0576984
  -0.04483978]
 [-0.04483978 -0.04483978 -0.04483978 ... -0.04483978 -0.04483978
  -0.04483978]
 [-0.02506754 -0.03861942 -0.00492718 ... -0.03486478 -0.02448526
  -0.04483978]
 ...
 [ 0.01171863 -0.03836875 -0.01656688 ... -0.04483978 -0.02158436
  -0.04483978]
 [ 0.01604981  0.14979033  0.1

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),


[[ 0.06776802  0.33844322  0.8981394  ... -0.39352173 -0.04206014
  -0.15834984]
 [-0.23612042 -0.32505122 -0.39352173 ... -0.39352173 -0.39352173
  -0.39352173]
 [ 0.9581345   1.5222803   1.4131235  ...  0.49931556  0.08543355
  -0.05479839]
 ...
 [-0.2601557  -0.11628606 -0.2621123  ... -0.1992561  -0.39352173
  -0.39352173]
 [-0.25419757 -0.17706664 -0.31109    ... -0.39352173 -0.2804896
  -0.39352173]
 [-0.39352173 -0.39352173 -0.39352173 ... -0.39352173 -0.39352173
   0.3350548 ]]
[[-0.04665888 -0.00115393  0.00954696 ... -0.04665888 -0.04665888
  -0.04665888]
 [-0.04112972 -0.04665888 -0.04665888 ... -0.04665888 -0.04665888
  -0.04665888]
 [-0.03233843 -0.02904052  0.00776949 ... -0.01533623 -0.02979037
  -0.04665888]
 ...
 [-0.04665888 -0.03234728 -0.04665888 ... -0.03581723 -0.04665888
  -0.04665888]
 [-0.04665888 -0.03932735 -0.01559491 ... -0.04665888  0.00159981
  -0.04665888]
 [-0.04665888 -0.04665888 -0.04665888 ... -0.04665888 -0.04665888
  -0.03964435]]
[[ 0.09621675  0.

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/Evaluator.py:368: RuntimeWarning: invalid value encountered in double_scalars
  results_current_cutoff[EvaluatorMetrics.NDCG.value]                 += ndcg(recommended_items_current_cutoff, relevant_items, relevance=self.get_user_test_ratings(test_user), at=cutoff)


[[-3.1936997e-01 -3.1936997e-01 -3.1936997e-01 ... -3.1936997e-01
  -3.1936997e-01 -3.1936997e-01]
 [-2.4662912e-01 -2.2820710e-01 -2.8434062e-01 ... -1.5533257e-01
  -3.1936997e-01 -3.1936997e-01]
 [-1.3604827e-01  7.3522377e-01 -1.3481265e-02 ... -3.1936997e-01
  -3.1936997e-01 -3.1936997e-01]
 ...
 [ 8.5305445e-02  6.7290189e-03 -3.1936997e-01 ... -3.1936997e-01
   1.8214503e-01  3.6773810e+00]
 [-3.1936997e-01 -3.1936997e-01 -1.9387285e-01 ... -3.1936997e-01
  -3.1936997e-01 -3.1936997e-01]
 [-2.5116774e-01 -2.3586597e-01  9.2294917e-04 ... -3.1936997e-01
  -1.7196946e-01 -3.1936997e-01]]
[[-3.5684261e-02 -3.5684261e-02 -3.5684261e-02 ... -3.5684261e-02
  -3.5684261e-02 -3.5684261e-02]
 [-3.5684261e-02 -3.5684261e-02 -3.5684261e-02 ...  2.4192074e-02
  -3.5684261e-02 -3.5684261e-02]
 [-2.4509236e-02  1.0482927e-01 -3.6426901e-03 ... -3.5684261e-02
  -3.5684261e-02 -3.5684261e-02]
 ...
 [-2.4980880e-02 -3.5684261e-02 -3.5684261e-02 ... -3.5684261e-02
   2.4871875e-05 -2.4079813e-02]

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.051713,0.090856,0.085611,0.026183,0.044847,0.170983,NaN,0.064478,0.333841,0.209599,...,0.961829,0.321098,0.961829,0.039172,9.152272,0.992204,0.077145,0.678285,4.27221,0.26411


In [3]:
#1
# BEST MODEL UNTIL NOW - CONTROPROVA
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

Similarity column 24507 (100.0%), 6107.44 column/sec. Elapsed time 4.01 sec
Similarity column 41629 (100.0%), 2506.20 column/sec. Elapsed time 16.61 sec
[[-0.09095383 -0.24738513 -0.10779274 ... -0.350219   -0.06725492
  -0.350219  ]
 [-0.350219   -0.350219   -0.350219   ... -0.350219   -0.350219
  -0.350219  ]
 [ 0.69090515  0.50116843  0.70487016 ... -0.15452862 -0.0322994
  -0.28700966]
 ...
 [ 0.07370103 -0.2261711  -0.03897558 ... -0.350219   -0.21208452
  -0.350219  ]
 [ 0.7203803   0.47170183  0.4486557  ... -0.33649695  0.32763025
  -0.00881661]
 [ 0.715159    0.7706233   0.6208966  ... -0.05203089  0.21328293
  -0.02477284]]
[[-0.04483978 -0.03094017  0.00948494 ... -0.04483978  0.0576984
  -0.04483978]
 [-0.04483978 -0.04483978 -0.04483978 ... -0.04483978 -0.04483978
  -0.04483978]
 [-0.02506754 -0.03861942 -0.00492718 ... -0.03486478 -0.02448526
  -0.04483978]
 ...
 [ 0.01171863 -0.03836875 -0.01656688 ... -0.04483978 -0.02158436
  -0.04483978]
 [ 0.01604981  0.14979033  0.1

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),


[[ 0.06776802  0.33844322  0.8981394  ... -0.39352173 -0.04206014
  -0.15834984]
 [-0.23612042 -0.32505122 -0.39352173 ... -0.39352173 -0.39352173
  -0.39352173]
 [ 0.9581345   1.5222803   1.4131235  ...  0.49931556  0.08543355
  -0.05479839]
 ...
 [-0.2601557  -0.11628606 -0.2621123  ... -0.1992561  -0.39352173
  -0.39352173]
 [-0.25419757 -0.17706664 -0.31109    ... -0.39352173 -0.2804896
  -0.39352173]
 [-0.39352173 -0.39352173 -0.39352173 ... -0.39352173 -0.39352173
   0.3350548 ]]
[[-0.04665888 -0.00115393  0.00954696 ... -0.04665888 -0.04665888
  -0.04665888]
 [-0.04112972 -0.04665888 -0.04665888 ... -0.04665888 -0.04665888
  -0.04665888]
 [-0.03233843 -0.02904052  0.00776949 ... -0.01533623 -0.02979037
  -0.04665888]
 ...
 [-0.04665888 -0.03234728 -0.04665888 ... -0.03581723 -0.04665888
  -0.04665888]
 [-0.04665888 -0.03932735 -0.01559491 ... -0.04665888  0.00159981
  -0.04665888]
 [-0.04665888 -0.04665888 -0.04665888 ... -0.04665888 -0.04665888
  -0.03964435]]
[[ 0.09621675  0.

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/Evaluator.py:368: RuntimeWarning: invalid value encountered in double_scalars
  results_current_cutoff[EvaluatorMetrics.NDCG.value]                 += ndcg(recommended_items_current_cutoff, relevant_items, relevance=self.get_user_test_ratings(test_user), at=cutoff)


[[-3.1936997e-01 -3.1936997e-01 -3.1936997e-01 ... -3.1936997e-01
  -3.1936997e-01 -3.1936997e-01]
 [-2.4662912e-01 -2.2820710e-01 -2.8434062e-01 ... -1.5533257e-01
  -3.1936997e-01 -3.1936997e-01]
 [-1.3604827e-01  7.3522377e-01 -1.3481265e-02 ... -3.1936997e-01
  -3.1936997e-01 -3.1936997e-01]
 ...
 [ 8.5305445e-02  6.7290189e-03 -3.1936997e-01 ... -3.1936997e-01
   1.8214503e-01  3.6773810e+00]
 [-3.1936997e-01 -3.1936997e-01 -1.9387285e-01 ... -3.1936997e-01
  -3.1936997e-01 -3.1936997e-01]
 [-2.5116774e-01 -2.3586597e-01  9.2294917e-04 ... -3.1936997e-01
  -1.7196946e-01 -3.1936997e-01]]
[[-3.5684261e-02 -3.5684261e-02 -3.5684261e-02 ... -3.5684261e-02
  -3.5684261e-02 -3.5684261e-02]
 [-3.5684261e-02 -3.5684261e-02 -3.5684261e-02 ...  2.4192074e-02
  -3.5684261e-02 -3.5684261e-02]
 [-2.4509236e-02  1.0482927e-01 -3.6426901e-03 ... -3.5684261e-02
  -3.5684261e-02 -3.5684261e-02]
 ...
 [-2.4980880e-02 -3.5684261e-02 -3.5684261e-02 ... -3.5684261e-02
   2.4871875e-05 -2.4079813e-02]

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.051713,0.090856,0.085611,0.026183,0.044847,0.170983,NaN,0.064478,0.333841,0.209599,...,0.961829,0.321098,0.961829,0.039172,9.152272,0.992204,0.077145,0.678285,4.27221,0.26411


In [4]:
#1
# BEST MODEL UNTIL NOW - CONTROPROVA
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation2.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

Similarity column 24507 (100.0%), 6125.08 column/sec. Elapsed time 4.00 sec
Similarity column 41629 (100.0%), 2523.51 column/sec. Elapsed time 16.50 sec
[[-0.09095383 -0.24738513 -0.10779274 ... -0.350219   -0.06725492
  -0.350219  ]
 [-0.350219   -0.350219   -0.350219   ... -0.350219   -0.350219
  -0.350219  ]
 [ 0.69090515  0.50116843  0.70487016 ... -0.15452862 -0.0322994
  -0.28700966]
 ...
 [ 0.07370103 -0.2261711  -0.03897558 ... -0.350219   -0.21208452
  -0.350219  ]
 [ 0.7203803   0.47170183  0.4486557  ... -0.33649695  0.32763025
  -0.00881661]
 [ 0.715159    0.7706233   0.6208966  ... -0.05203089  0.21328293
  -0.02477284]]
[[-0.04483978 -0.03094017  0.00948494 ... -0.04483978  0.0576984
  -0.04483978]
 [-0.04483978 -0.04483978 -0.04483978 ... -0.04483978 -0.04483978
  -0.04483978]
 [-0.02506754 -0.03861942 -0.00492718 ... -0.03486478 -0.02448526
  -0.04483978]
 ...
 [ 0.01171863 -0.03836875 -0.01656688 ... -0.04483978 -0.02158436
  -0.04483978]
 [ 0.01604981  0.14979033  0.1

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),


[[-0.3769829   0.44951    -0.3769829  ... -0.08949474 -0.3769829
  -0.3276711 ]
 [-0.29737777 -0.3769829  -0.16272573 ... -0.11575697 -0.3769829
  -0.3769829 ]
 [-0.08236984  0.29139146 -0.18056837 ... -0.16381419 -0.1650109
  -0.08076361]
 ...
 [-0.08082099  0.03114429 -0.1925043  ... -0.3674302  -0.35146928
  -0.36053443]
 [-0.1650918  -0.0026046   0.01418216 ... -0.02902653 -0.13982306
  -0.3158482 ]
 [ 0.9682746   1.0494757   1.5036074  ... -0.28835484  0.5853197
   0.08385601]]
[[-0.04761218 -0.00377064 -0.04761218 ... -0.01724878 -0.04761218
  -0.04761218]
 [-0.04761218 -0.04761218 -0.034616   ... -0.04761218 -0.04761218
  -0.04761218]
 [-0.04761218 -0.04231776 -0.04320335 ... -0.04357129 -0.02973216
  -0.04761218]
 ...
 [-0.01853678 -0.00967317 -0.04761218 ... -0.04761218 -0.0399926
  -0.03663491]
 [-0.04761218 -0.03849411 -0.0430162  ... -0.03942018 -0.03977643
  -0.04761218]
 [ 0.0279929   0.06467414  0.1136465  ... -0.02799734  0.00446491
  -0.04761218]]
[[-0.22659001  0.0889

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


[[ 0.06776802  0.33844322  0.8981394  ... -0.39352173 -0.04206014
  -0.15834984]
 [-0.23612042 -0.32505122 -0.39352173 ... -0.39352173 -0.39352173
  -0.39352173]
 [ 0.9581345   1.5222803   1.4131235  ...  0.49931556  0.08543355
  -0.05479839]
 ...
 [-0.2601557  -0.11628606 -0.2621123  ... -0.1992561  -0.39352173
  -0.39352173]
 [-0.25419757 -0.17706664 -0.31109    ... -0.39352173 -0.2804896
  -0.39352173]
 [-0.39352173 -0.39352173 -0.39352173 ... -0.39352173 -0.39352173
   0.3350548 ]]
[[-0.04665888 -0.00115393  0.00954696 ... -0.04665888 -0.04665888
  -0.04665888]
 [-0.04112972 -0.04665888 -0.04665888 ... -0.04665888 -0.04665888
  -0.04665888]
 [-0.03233843 -0.02904052  0.00776949 ... -0.01533623 -0.02979037
  -0.04665888]
 ...
 [-0.04665888 -0.03234728 -0.04665888 ... -0.03581723 -0.04665888
  -0.04665888]
 [-0.04665888 -0.03932735 -0.01559491 ... -0.04665888  0.00159981
  -0.04665888]
 [-0.04665888 -0.04665888 -0.04665888 ... -0.04665888 -0.04665888
  -0.03964435]]
[[ 0.09621675  0.

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.051713,0.090856,0.085611,0.026183,0.044847,0.170983,NaN,0.064478,0.333841,0.209599,...,0.961829,0.321098,0.961829,0.039172,9.152272,0.992204,0.077145,0.678285,4.27221,0.26411


In [5]:
#1
# BEST MODEL UNTIL NOW - CONTROPROVA
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train2)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation2.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

Similarity column 24507 (100.0%), 5880.23 column/sec. Elapsed time 4.17 sec
Similarity column 41629 (100.0%), 2426.38 column/sec. Elapsed time 17.16 sec
[[-0.09095383 -0.24738513 -0.10779274 ... -0.350219   -0.06725492
  -0.350219  ]
 [-0.350219   -0.350219   -0.350219   ... -0.350219   -0.350219
  -0.350219  ]
 [ 0.69090515  0.50116843  0.70487016 ... -0.15452862 -0.0322994
  -0.28700966]
 ...
 [ 0.07370103 -0.2261711  -0.03897558 ... -0.350219   -0.21208452
  -0.350219  ]
 [ 0.7203803   0.47170183  0.4486557  ... -0.33649695  0.32763025
  -0.00881661]
 [ 0.715159    0.7706233   0.6208966  ... -0.05203089  0.21328293
  -0.02477284]]
[[-0.04483978 -0.03094017  0.00948494 ... -0.04483978  0.0576984
  -0.04483978]
 [-0.04483978 -0.04483978 -0.04483978 ... -0.04483978 -0.04483978
  -0.04483978]
 [-0.02506754 -0.03861942 -0.00492718 ... -0.03486478 -0.02448526
  -0.04483978]
 ...
 [ 0.01171863 -0.03836875 -0.01656688 ... -0.04483978 -0.02158436
  -0.04483978]
 [ 0.01604981  0.14979033  0.1

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),


[[-0.3769829   0.44951    -0.3769829  ... -0.08949474 -0.3769829
  -0.3276711 ]
 [-0.29737777 -0.3769829  -0.16272573 ... -0.11575697 -0.3769829
  -0.3769829 ]
 [-0.08236984  0.29139146 -0.18056837 ... -0.16381419 -0.1650109
  -0.08076361]
 ...
 [-0.08082099  0.03114429 -0.1925043  ... -0.3674302  -0.35146928
  -0.36053443]
 [-0.1650918  -0.0026046   0.01418216 ... -0.02902653 -0.13982306
  -0.3158482 ]
 [ 0.9682746   1.0494757   1.5036074  ... -0.28835484  0.5853197
   0.08385601]]
[[-0.04761218 -0.00377064 -0.04761218 ... -0.01724878 -0.04761218
  -0.04761218]
 [-0.04761218 -0.04761218 -0.034616   ... -0.04761218 -0.04761218
  -0.04761218]
 [-0.04761218 -0.04231776 -0.04320335 ... -0.04357129 -0.02973216
  -0.04761218]
 ...
 [-0.01853678 -0.00967317 -0.04761218 ... -0.04761218 -0.0399926
  -0.03663491]
 [-0.04761218 -0.03849411 -0.0430162  ... -0.03942018 -0.03977643
  -0.04761218]
 [ 0.0279929   0.06467414  0.1136465  ... -0.02799734  0.00446491
  -0.04761218]]
[[-0.22659001  0.0889

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


[[ 0.06776802  0.33844322  0.8981394  ... -0.39352173 -0.04206014
  -0.15834984]
 [-0.23612042 -0.32505122 -0.39352173 ... -0.39352173 -0.39352173
  -0.39352173]
 [ 0.9581345   1.5222803   1.4131235  ...  0.49931556  0.08543355
  -0.05479839]
 ...
 [-0.2601557  -0.11628606 -0.2621123  ... -0.1992561  -0.39352173
  -0.39352173]
 [-0.25419757 -0.17706664 -0.31109    ... -0.39352173 -0.2804896
  -0.39352173]
 [-0.39352173 -0.39352173 -0.39352173 ... -0.39352173 -0.39352173
   0.3350548 ]]
[[-0.04665888 -0.00115393  0.00954696 ... -0.04665888 -0.04665888
  -0.04665888]
 [-0.04112972 -0.04665888 -0.04665888 ... -0.04665888 -0.04665888
  -0.04665888]
 [-0.03233843 -0.02904052  0.00776949 ... -0.01533623 -0.02979037
  -0.04665888]
 ...
 [-0.04665888 -0.03234728 -0.04665888 ... -0.03581723 -0.04665888
  -0.04665888]
 [-0.04665888 -0.03932735 -0.01559491 ... -0.04665888  0.00159981
  -0.04665888]
 [-0.04665888 -0.04665888 -0.04665888 ... -0.04665888 -0.04665888
  -0.03964435]]
[[ 0.09621675  0.

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.051713,0.090856,0.085611,0.026183,0.044847,0.170983,NaN,0.064478,0.333841,0.209599,...,0.961829,0.321098,0.961829,0.039172,9.152272,0.992204,0.077145,0.678285,4.27221,0.26411
